## **Single-Runway Constraint Programming Model**

<h4 style="color:#0033a0">Parameters</h4>




\begin{aligned}
p & \quad \text{Number of aircraft} \\
E_i & \quad \text{Earliest landing time for aircraft } i \\
T_i & \quad \text{Target (preferred) landing time for aircraft } i \\
L_i & \quad \text{Latest landing time for aircraft } i \\
e_i & \quad \text{Penalty cost per unit time if } t_i < T_i \ ( \text{earliness cost}) \\
l_i & \quad \text{Penalty cost per unit time if } t_i > T_i \ ( \text{lateness cost}) \\
S_{i,j} & \quad \text{Separation time if aircraft } i \text{ lands immediately before } j \\
\end{aligned}

<br/> <h4 style="color:#0033a0">Decision Variables</h4>

\begin{aligned}
\mathit{pos}_i 
  &\quad \text{Position of aircraft } i \text{ in the landing sequence} 
   & (\mathit{pos}_i \in \{0, \dots, p-1\}) 
\\
t_i 
  &\quad \text{Landing time of aircraft } i 
   & (t_i \ge 0)
\\
E'_i 
  &\quad \text{Earliness of aircraft } i
   & (E'_i \ge 0)
\\
L'_i 
  &\quad \text{Lateness of aircraft } i
   & (L'_i \ge 0)
\\
\end{aligned}



<br/> <h4 style="color:#0033a0">Objective Function and Constraints</h4>

\begin{aligned}
&\textbf{Objective:} 
&& \min \quad \sum_{i=1}^{p} \Bigl(e_i\,E'_i + l_i\,L'_i\Bigr) \\[10pt]
&\textbf{Subject to:}
\\[5pt]
& \mathit{pos}_i \neq \mathit{pos}_j 
&& \forall i \neq j 
\quad \text{(all-different positions)} 
\\[5pt]
& E_i \;\le\; t_i \;\le\; L_i 
&& \forall i 
\\[5pt]
& E'_i = \max\{0,\;T_i - t_i\}, 
\quad L'_i = \max\{0,\;t_i - T_i\}
&& \forall i 
\\[5pt]
& \text{If } \mathit{pos}_i < \mathit{pos}_j 
   \text{ then } t_j \;\ge\; t_i + S_{i,j},
&& \forall i \neq j 
\\[3pt]
& \text{If } \mathit{pos}_j < \mathit{pos}_i 
   \text{ then } t_i \;\ge\; t_j + S_{j,i},
&& \forall i \neq j 
\\[5pt]
& \mathit{pos}_i \in \{0,\dots,p-1\}, 
\; t_i,\,E'_i,\,L'_i \ge 0
\quad \forall i 
\end{aligned}



<br/> <h4 style="color:#0033a0">SOLVING</h4>

In [21]:
def read_data_from_file(filename):
    """
    Reads data from a file with the specified format for an air traffic scheduling problem.

    Args:
        filename (str): The path to the data file.

    Returns:
        tuple: A tuple containing the parsed data:
            - num_planes (int): The number of planes.
            - freeze_time (int): The freeze time.
            - planes_data (list): A list of dictionaries, where each dictionary contains
              the data for a plane.
            - separation_times (list of lists): A 2D list of separation times.
    """
    try:
        with open(filename, "r") as f:
            # Read the first line: number of planes and freeze time
            first_line = f.readline().strip().split()
            num_planes = int(first_line[0])
            freeze_time = int(first_line[1])

            planes_data = []
            separation_times = []
            
            for _ in range(num_planes):
                line = f.readline().strip().split()
                appearance_time = int(line[0])
                earliest_landing_time = int(line[1])
                target_landing_time = int(line[2])
                latest_landing_time = int(line[3])
                penalty_early = float(line[4])
                penalty_late = float(line[5])
                planes_data.append(
                    {
                        "appearance_time": appearance_time,
                        "earliest_landing_time": earliest_landing_time,
                        "target_landing_time": target_landing_time,
                        "latest_landing_time": latest_landing_time,
                        "penalty_early": penalty_early,
                        "penalty_late": penalty_late,
                    }
                )
                
                separation_row = []
                while len(separation_row) < num_planes:
                    line = f.readline().strip().split()
                    separation_row.extend([int(x) for x in line])

                separation_times.append(separation_row)

        return num_planes, freeze_time, planes_data, separation_times

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None, None, None, None
    except ValueError:
        print(f"Error: Error reading data in file '{filename}'.")
        return None, None, None, None

In [22]:
filename = "data/airland4.txt"

num_planes, freeze_time, planes_data, separation_times = read_data_from_file(filename)

if num_planes is not None:
    print("Number of planes:", num_planes)
    print("Freeze time:", freeze_time)
    print("\nPlane data:")
    for i, plane in enumerate(planes_data):
        print(f"Plane {i+1}: {plane}")
    print("\nSeparation times:")
    for i, row in enumerate(separation_times):
        print(f"After plane {i+1}: {row}")

Number of planes: 20
Freeze time: 35

Plane data:
Plane 1: {'appearance_time': 7, 'earliest_landing_time': 82, 'target_landing_time': 92, 'latest_landing_time': 510, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 2: {'appearance_time': 9, 'earliest_landing_time': 84, 'target_landing_time': 93, 'latest_landing_time': 509, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 3: {'appearance_time': 75, 'earliest_landing_time': 150, 'target_landing_time': 183, 'latest_landing_time': 599, 'penalty_early': 10.0, 'penalty_late': 10.0}
Plane 4: {'appearance_time': 129, 'earliest_landing_time': 204, 'target_landing_time': 270, 'latest_landing_time': 760, 'penalty_early': 10.0, 'penalty_late': 10.0}
Plane 5: {'appearance_time': 14, 'earliest_landing_time': 89, 'target_landing_time': 98, 'latest_landing_time': 510, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 6: {'appearance_time': 29, 'earliest_landing_time': 104, 'target_landing_time': 117, 'latest_landing_time': 552, 'penalty_early': 3

In [23]:
from ortools.sat.python import cp_model

def create_or_tools_cp_model_with_permutation(num_planes, freeze_time, planes_data, separation_times):
    """
    Creates an OR-Tools CP-SAT model (and variables) for the single-runway 
    aircraft landing problem using a 'permutation' approach to avoid big-M constraints.
    
    Args:
        num_planes (int): Number of planes.
        freeze_time (int): Freeze time (not used here, can be integrated if needed).
        planes_data (list[dict]): For each plane i:
            {
                'appearance_time': int,
                'earliest_landing_time': int,
                'target_landing_time': int,
                'latest_landing_time': int,
                'penalty_early': float,
                'penalty_late': float
            }
        separation_times (list[list]): S[i][j] = min. separation if plane i lands before j.

    Returns:
        model (cp_model.CpModel),
        variables (dict): {
            "position": [IntVar,...],
            "landing_time": [IntVar,...],
            "earliness": [IntVar,...],
            "lateness": [IntVar,...]
        }
    """
    model = cp_model.CpModel()

    # Extract convenience arrays
    E = [p["earliest_landing_time"] for p in planes_data]
    T = [p["target_landing_time"]   for p in planes_data]
    L = [p["latest_landing_time"]   for p in planes_data]
    cost_e = [p["penalty_early"]    for p in planes_data]
    cost_l = [p["penalty_late"]     for p in planes_data]

    # --- 1) Define Variables ---

    # position[i] in [0..num_planes-1], says which "slot" plane i occupies
    position = [
        model.NewIntVar(0, num_planes - 1, f"position_{i}") 
        for i in range(num_planes)
    ]

    # AllDifferent to ensure we have a permutation of planes
    model.AddAllDifferent(position)

    # landing_time[i] in [0..someLargeNumber]
    # must also respect earliest/latest constraints (we'll add those next)
    landing_time = [
        model.NewIntVar(0, 10000000, f"landing_time_{i}") 
        for i in range(num_planes)
    ]

    # earliness / lateness
    earliness = [
        model.NewIntVar(0,
            max(
                planes_data[i]["target_landing_time"]
                - planes_data[i]["earliest_landing_time"],
                0,
            ),        
            f"earliness_{i}")
        for i in range(num_planes)
    ]
    lateness = [
        model.NewIntVar(0,
            max(
                planes_data[i]["latest_landing_time"]
                - planes_data[i]["target_landing_time"],
                0,
            ), f"lateness_{i}")
        for i in range(num_planes)
    ]

    # --- 2) Time Window Constraints ---
    for i in range(num_planes):
        # earliest <= landing_time[i] <= latest
        model.Add(landing_time[i] >= E[i])
        model.Add(landing_time[i] <= L[i])

    # --- 3) Earliness / Lateness definitions ---
    for i in range(num_planes):
        # earliness[i] = max(0, T[i] - landing_time[i])
        # lateness[i]  = max(0, landing_time[i] - T[i])
        model.Add(earliness[i] >= T[i] - landing_time[i])
        model.Add(earliness[i] >= 0)
        model.Add(lateness[i]  >= landing_time[i] - T[i])
        model.Add(lateness[i]  >= 0)

    # --- 4) Separation Constraints via Permutation ---

    # We want: if position[i] < position[j] => landing_time[j] >= landing_time[i] + S[i][j]
    # In CP-SAT, we can't directly write "if position[i] < position[j]", 
    # so we'll define a Boolean reification for "position[i] < position[j]".
    # Then we'll OnlyEnforceIf that Boolean is True or False.
    #
    # We'll do this for each unordered pair (i<j).
    for i in range(num_planes):
        for j in range(i+1, num_planes):
            # Boolean var: iBeforeJ = (position[i] < position[j])
            iBeforeJ = model.NewBoolVar(f"iBeforeJ_{i}_{j}")

            # Constrain (position[i] < position[j]) <=> iBeforeJ = 1
            model.Add(position[i] < position[j]).OnlyEnforceIf(iBeforeJ)
            model.Add(position[i] >= position[j]).OnlyEnforceIf(iBeforeJ.Not())

            # If iBeforeJ then t[j] >= t[i] + S[i][j]
            model.Add(landing_time[j] >= landing_time[i] + separation_times[i][j])\
                 .OnlyEnforceIf(iBeforeJ)

            # If jBeforeI then t[i] >= t[j] + S[j][i]
            model.Add(landing_time[i] >= landing_time[j] + separation_times[j][i])\
                 .OnlyEnforceIf(iBeforeJ.Not())

    # --- 5) Objective: minimize sum of earliness + lateness cost ---
    cost_terms = []
    for i in range(num_planes):
        cost_terms.append(cost_e[i] * earliness[i])
        cost_terms.append(cost_l[i] * lateness[i])
    model.Minimize(sum(cost_terms))

    # --- 6) Return the model and variables ---
    variables = {
        "position": position,
        "landing_time": landing_time,
        "earliness": earliness,
        "lateness": lateness,
    }
    return model, variables


def solve_cp_model_with_permutation(num_planes, freeze_time, planes_data, separation_times):
    """Builds and solves the single-runway CP model with a permutation approach."""
    model, vars_ = create_or_tools_cp_model_with_permutation(
        num_planes, freeze_time, planes_data, separation_times
    )

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        print("Status:", solver.StatusName(status))
        print("Objective (total earliness + lateness):", solver.ObjectiveValue())
        print()

        position = vars_["position"]
        landing_time = vars_["landing_time"]
        earliness = vars_["earliness"]
        lateness = vars_["lateness"]

        # Print results
        #  - plane i in position solver.Value(position[i])
        #  - landing time
        for i in range(num_planes):
            pos_val = solver.Value(position[i])
            t_val   = solver.Value(landing_time[i])
            e_      = solver.Value(earliness[i])
            L_      = solver.Value(lateness[i])
            T_      = planes_data[i]["target_landing_time"]
            print(f"Plane {i}: position={pos_val}, landing={t_val}, "
                  f"E'={e_}, L'={L_}, target={T_}")

        # Sort planes by their actual position to see the schedule order
        print("\nSchedule order (by position):")
        schedule = sorted(range(num_planes), key=lambda i: solver.Value(position[i]))
        for k in schedule:
            print(f" -> Plane {k} (pos={solver.Value(position[k])}, land={solver.Value(landing_time[k])})")

    else:
        print("No feasible/optimal solution found. Status:", solver.StatusName(status))


if __name__ == "__main__":

    if num_planes is not None:
        solve_cp_model_with_permutation(num_planes, freeze_time, planes_data, separation_times)


Status: OPTIMAL
Objective (total earliness + lateness): 2520.0

Plane 0: position=0, landing=82, E'=10, L'=0, target=92
Plane 1: position=1, landing=90, E'=3, L'=0, target=93
Plane 2: position=14, landing=201, E'=0, L'=18, target=183
Plane 3: position=15, landing=270, E'=0, L'=0, target=270
Plane 4: position=2, landing=98, E'=0, L'=0, target=98
Plane 5: position=6, landing=130, E'=0, L'=13, target=117
Plane 6: position=5, landing=122, E'=0, L'=4, target=118
Plane 7: position=4, landing=114, E'=2, L'=0, target=116
Plane 8: position=3, landing=106, E'=6, L'=0, target=112
Plane 9: position=16, landing=280, E'=0, L'=0, target=280
Plane 10: position=18, landing=295, E'=0, L'=0, target=295
Plane 11: position=8, landing=146, E'=10, L'=0, target=156
Plane 12: position=7, landing=138, E'=0, L'=1, target=137
Plane 13: position=17, landing=291, E'=0, L'=0, target=291
Plane 14: position=13, landing=186, E'=0, L'=12, target=174
Plane 15: position=9, landing=154, E'=2, L'=0, target=156
Plane 16: pos

## **Multiple-Runway Constraint Programming Model**

<h4 style="color:#0033a0">Parameters</h4>

\begin{aligned}
p & \quad \text{Number of aircraft (indexed by } i = 1,\dots,p\text{)}\\
R & \quad \text{Number of available runways (indexed by } r = 1,\dots,R\text{)}\\
E_i & \quad \text{Earliest landing time allowed for aircraft } i\\
T_i & \quad \text{Target (preferred) landing time for aircraft } i\\
L_i & \quad \text{Latest landing time allowed for aircraft } i\\
e_i & \quad \text{Penalty cost per unit time if } t_i < T_i \ ( \text{earliness cost})\\
l_i & \quad \text{Penalty cost per unit time if } t_i > T_i \ ( \text{lateness cost})\\
S_{i,j} & \quad \text{Separation time if aircraft } i \text{ lands before } j \\
\end{aligned}

<br/> <h4 style="color:#0033a0">Decision Variables</h4>

\begin{aligned}
r_i \quad &\in \{1,\dots,R\}
&& \text{Runway choice for aircraft } i \\
\mathit{pos}_i \quad &\in \{0,\dots,p-1\}
&& \text{Landing order slot of aircraft } i \text{ on its chosen runway}\\
t_i \quad &\ge 0
&& \text{Landing time of aircraft } i\\
E'_i \quad &\ge 0
&& \text{Earliness of aircraft } i \quad (E'_i = \max\{0,T_i - t_i\})\\
L'_i \quad &\ge 0
&& \text{Lateness of aircraft } i \quad (L'_i = \max\{0,t_i - T_i\})
\end{aligned}

<br/> <h4 style="color:#0033a0">Objective Function and Constraints</h4>

\begin{aligned}
& \textbf{Objective:} 
\quad \min \; \sum_{i=1}^p \Bigl(e_i\,E'_i + l_i\,L'_i\Bigr) \\[8pt]
& \textbf{Subject to:} \\[6pt]
& \text{(1) Time windows:} \quad E_i \;\le\; t_i \;\le\; L_i 
&& \forall i \\[4pt]
& \text{(2) Earliness/Lateness:} \quad
  E'_i \;=\;\max\{\,0,\;T_i - t_i\}, 
  \quad
  L'_i \;=\;\max\{\,0,\;t_i - T_i\}
&& \forall i \\[4pt]
& \text{(3) Separation constraints:} \\
& \quad \text{If } r_i = r_j \text{ and } \mathit{pos}_i < \mathit{pos}_j,
    \quad \Longrightarrow\; 
    t_j \;\ge\; t_i + S_{i,j}
&& \forall i \neq j \\[2pt]
& \quad \text{If } r_i = r_j \text{ and } \mathit{pos}_j < \mathit{pos}_i,
    \quad \Longrightarrow\;
    t_i \;\ge\; t_j + S_{j,i}
&& \forall i \neq j \\[4pt]
& \text{(4) Distinct positions on the same runway:} \\
& \quad \text{If } r_i = r_j, \quad \mathit{pos}_i \neq \mathit{pos}_j,
&& \forall i \neq j \\[4pt]
& \text{(5) Domain constraints:} \\
& \quad r_i \in \{1,\dots,R\}, \quad
          \mathit{pos}_i \in \{0,\dots,p-1\}, \quad
          t_i \ge 0, \\
& \quad E'_i,L'_i \ge 0, \quad \forall i
\end{aligned}


In [24]:
from ortools.sat.python import cp_model

def create_or_tools_cp_model_multi_runway(num_planes, num_runways, freeze_time, planes_data, separation_times):
    """
    Creates an OR-Tools CP-SAT model for a multi-runway aircraft landing problem,
    extending a 'permutation approach' so that each plane i has:
      - runway[i]: the runway it uses
      - position[i]: the order among planes sharing that same runway
      - landing_time[i], earliness[i], lateness[i], etc.

    Args:
        num_planes (int): Number of planes.
        num_runways (int): Number of runways available.
        planes_data (list[dict]): For each plane i, containing:
            'earliest_landing_time', 'target_landing_time', 'latest_landing_time'
            'penalty_early', 'penalty_late', etc.
        separation_times (list[list]): S[i][j], the required separation if plane i lands before j
                                      on the same runway.

    Returns:
        model (cp_model.CpModel),
        variables (dict): {
            "runway": list of IntVar,
            "position": list of IntVar,
            "landing_time": list of IntVar,
            "earliness": list of IntVar,
            "lateness": list of IntVar
        }
    """
    model = cp_model.CpModel()

    # Extract convenience arrays for times & costs
    E = [p["earliest_landing_time"] for p in planes_data]  # earliest time
    T = [p["target_landing_time"]   for p in planes_data]  # target time
    L = [p["latest_landing_time"]   for p in planes_data]  # latest time
    cost_e = [p["penalty_early"]    for p in planes_data]  # e_i
    cost_l = [p["penalty_late"]     for p in planes_data]  # l_i

    # 1) Decision Variables

    # runway[i] in [0..num_runways-1]
    runway = [
        model.NewIntVar(0, num_runways - 1, f"runway_{i}")
        for i in range(num_planes)
    ]

    # position[i] in [0..num_planes-1]
    # We'll only force distinct positions among planes on the same runway
    position = [
        model.NewIntVar(0, num_planes - 1, f"position_{i}")
        for i in range(num_planes)
    ]

    # landing_time[i], within a broad feasible range
    landing_time = [
        model.NewIntVar(0, 9999999, f"landing_time_{i}")
        for i in range(num_planes)
    ]

    # earliness[i], lateness[i]
    earliness = []
    lateness = []
    for i in range(num_planes):
        e_ = model.NewIntVar(0, 9999999, f"earliness_{i}")
        l_ = model.NewIntVar(0, 9999999, f"lateness_{i}")
        earliness.append(e_)
        lateness.append(l_)

    # 2) Time Window Constraints: E[i] <= landing_time[i] <= L[i]
    for i in range(num_planes):
        model.Add(landing_time[i] >= E[i])
        model.Add(landing_time[i] <= L[i])

    # 3) Earliness / Lateness definitions:
    for i in range(num_planes):
        # E'[i] = max(0, T[i] - t[i])
        model.Add(earliness[i] >= T[i] - landing_time[i])
        model.Add(earliness[i] >= 0)
        # L'[i] = max(0, t[i] - T[i])
        model.Add(lateness[i] >= landing_time[i] - T[i])
        model.Add(lateness[i] >= 0)

    # 4) Enforce distinct positions only for planes on the same runway
    #
    # For each pair of planes (i < j), define a Boolean same_runway[i,j] to indicate
    # if plane i and j share the same runway. If they do, they must have distinct
    # positions, and a separation constraint is enforced based on who is first.
    same_runway = {}
    iBeforeJ = {}

    for i in range(num_planes):
        for j in range(i + 1, num_planes):
            # same_runway[i,j] => runway[i] == runway[j]
            same_runway[(i, j)] = model.NewBoolVar(f"same_runway_{i}_{j}")
            model.Add(runway[i] == runway[j]).OnlyEnforceIf(same_runway[(i, j)])
            model.Add(runway[i] != runway[j]).OnlyEnforceIf(same_runway[(i, j)].Not())

            # If they are on the same runway, positions must differ
            model.Add(position[i] != position[j]).OnlyEnforceIf(same_runway[(i, j)])

            # iBeforeJ[i,j] => "plane i is before plane j on that runway"
            iBeforeJ[(i, j)] = model.NewBoolVar(f"iBeforeJ_{i}_{j}")

            # iBeforeJ => position[i] < position[j] AND same_runway[i,j]
            model.Add(position[i] < position[j]).OnlyEnforceIf(iBeforeJ[(i, j)])
            model.Add(position[i] >= position[j]).OnlyEnforceIf(iBeforeJ[(i, j)].Not())

            # also iBeforeJ => same_runway[i,j]
            model.Add(same_runway[(i, j)] == 1).OnlyEnforceIf(iBeforeJ[(i, j)])

            # 5) Separation constraints:
            # If same_runway = true and iBeforeJ = true => t[j] >= t[i] + S[i][j]
            model.Add(
                landing_time[j] >= landing_time[i] + separation_times[i][j]
            ).OnlyEnforceIf(iBeforeJ[(i, j)])
            model.Add(
                landing_time[i] >= landing_time[j] + separation_times[j][i]
            ).OnlyEnforceIf(iBeforeJ[(i, j)].Not()).OnlyEnforceIf(same_runway[(i, j)])
            # If same_runway = false, no separation constraint needed for i & j.

    # 6) Objective: minimize sum of earliness + lateness
    cost_terms = []
    for i in range(num_planes):
        cost_terms.append(cost_e[i] * earliness[i])
        cost_terms.append(cost_l[i] * lateness[i])
    model.Minimize(sum(cost_terms))

    variables = {
        "runway": runway,
        "position": position,
        "landing_time": landing_time,
        "earliness": earliness,
        "lateness": lateness,
    }

    return model, variables


def solve_cp_model_multi_runway(num_planes, num_runways,freeze_time, planes_data, separation_times):
    """
    Builds and solves the multi-runway CP model using a (runway_i, position_i) approach.
    """
    model, vars_ = create_or_tools_cp_model_multi_runway(
        num_planes, num_runways,freeze_time, planes_data, separation_times
    )

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        print("Status:", solver.StatusName(status))
        print("Objective (total earliness + lateness):", solver.ObjectiveValue())
        print()

        runway = vars_["runway"]
        position = vars_["position"]
        landing_time = vars_["landing_time"]
        earliness = vars_["earliness"]
        lateness = vars_["lateness"]

        # Print details per plane
        for i in range(num_planes):
            r_val = solver.Value(runway[i])
            pos_val = solver.Value(position[i])
            t_val = solver.Value(landing_time[i])
            e_ = solver.Value(earliness[i])
            L_ = solver.Value(lateness[i])
            print(f"Plane {i}: runway={r_val}, position={pos_val}, land={t_val}, "
                  f"E'={e_}, L'={L_}")

        # Print schedule per runway
        print("\nSchedule by runway:")
        for r in range(num_runways):
            # get planes on runway r
            planes_r = [i for i in range(num_planes) if solver.Value(runway[i]) == r]
            # sort them by position
            planes_r.sort(key=lambda i: solver.Value(position[i]))
            print(f"  Runway {r}:")
            for i in planes_r:
                print(f"     Plane {i}, pos={solver.Value(position[i])}, landing={solver.Value(landing_time[i])}")

    else:
        print("No feasible/optimal solution found. Status:", solver.StatusName(status))


In [25]:
if __name__ == "__main__":

    if num_planes is not None:
        solve_cp_model_multi_runway(num_planes,10, freeze_time, planes_data, separation_times)


Status: OPTIMAL
Objective (total earliness + lateness): 0.0

Plane 0: runway=6, position=11, land=92, E'=0, L'=0
Plane 1: runway=4, position=9, land=93, E'=0, L'=0
Plane 2: runway=4, position=10, land=183, E'=0, L'=0
Plane 3: runway=4, position=11, land=270, E'=0, L'=0
Plane 4: runway=3, position=8, land=98, E'=0, L'=0
Plane 5: runway=3, position=9, land=117, E'=0, L'=0
Plane 6: runway=8, position=8, land=118, E'=0, L'=0
Plane 7: runway=1, position=7, land=116, E'=0, L'=0
Plane 8: runway=2, position=2, land=112, E'=0, L'=0
Plane 9: runway=2, position=5, land=280, E'=0, L'=0
Plane 10: runway=2, position=7, land=295, E'=0, L'=0
Plane 11: runway=2, position=4, land=156, E'=0, L'=0
Plane 12: runway=2, position=3, land=137, E'=0, L'=0
Plane 13: runway=2, position=6, land=291, E'=0, L'=0
Plane 14: runway=5, position=2, land=174, E'=0, L'=0
Plane 15: runway=5, position=1, land=156, E'=0, L'=0
Plane 16: runway=7, position=1, land=170, E'=0, L'=0
Plane 17: runway=9, position=1, land=169, E'=0, 